In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [ ]:
import numpy as np
import random
from tqdm import tqdm
import zipfile, os, shutil
from google.colab import drive
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
print("1.Mounting Google Drive...")
drive.mount('/content/drive/')

1.Mounting Google Drive...
Mounted at /content/drive/


In [ ]:
zip_path = "/content/drive/MyDrive/archive.zip"
extract_dir = "/content/chest_xray"

In [ ]:
print(f"2. Extracting {zip_path}...")
os.makedirs(extract_dir, exist_ok=True)
try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print("Extraction successful.")
except FileNotFoundError:
    print(f"Error: ZIP file not found at {zip_path}. Please check the path.")
    exit()

2. Extracting /content/drive/MyDrive/archive.zip...
Extraction successful.


In [ ]:
data_dir = None
for root, dirs, files in os.walk(extract_dir):
    if "train" in dirs:
        data_dir = root
        break
if data_dir is None:
    data_dir = extract_dir
print("✅ Dataset base directory found at:", data_dir)

✅ Dataset base directory found at: /content/chest_xray/chest_xray


In [ ]:
print("\n3.  Preparing Data Loaders...")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
])


3.  Preparing Data Loaders...


In [ ]:
train_path = os.path.join(data_dir, "train")
try:
    dataset = ImageFolder(root=train_path, transform=transform)
    train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
except RuntimeError as e:
     print(f" Error loading data from {train_path}. Check if 'train' subfolder exists and contains images.")
     print(e)
     exit()

In [ ]:
print("4. 🧠 Extracting features using ResNet18...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Identity()  # remove final classification layer
resnet = resnet.to(device)
resnet.eval()

4. 🧠 Extracting features using ResNet18...
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 44.7M/44.7M [00:00<00:00, 173MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
features, labels = [], []

In [ ]:
with torch.no_grad():
    for imgs, lbls in tqdm(train_loader, desc="🔍 Feature Extraction Progress"):
        imgs = imgs.to(device)
        out = resnet(imgs).cpu().numpy()
        features.append(out)
        labels.append(lbls.numpy())

🔍 Feature Extraction Progress: 100%|██████████| 163/163 [01:41<00:00,  1.60it/s]


In [ ]:
features = np.concatenate(features)
labels = np.concatenate(labels)
print("Feature extraction complete! Shape:", features.shape)

Feature extraction complete! Shape: (5216, 512)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=42)
N_FEATURES = features.shape[1]
print(f"Data split: Train={len(X_train)}, Test={len(X_test)}. Total features: {N_FEATURES}")

Data split: Train=3912, Test=1304. Total features: 512


In [ ]:
# ==============================
# 🧬 Binary Particle Swarm Optimization (BPSO) - 10 ITERATION TEST
# ==============================

# --- Custom Hyperparameters for Quick Test ---
QUICK_N_ITERATIONS = 10  # Quick run limit
QUICK_N_PARTICLES = 50   # Swarm Size: A bit smaller for speed (50 instead of 100)
QUICK_W = 0.7            # Inertia weight: Kept at 0.7 for balance
QUICK_C1 = 1.7           # Cognitive coefficient
QUICK_C2 = 1.3           # Social coefficient
QUICK_V_MAX = 6.0        # Max velocity

# --- Initialization ---
N_FEATURES = features.shape[1] # Assumes 'features' is defined from step 4
positions = np.random.randint(0, 2, size=(QUICK_N_PARTICLES, N_FEATURES))
velocities = np.zeros((QUICK_N_PARTICLES, N_FEATURES))
pbest_positions = positions.copy()
pbest_fitness = np.full(QUICK_N_PARTICLES, -np.inf)
gbest_position = np.random.randint(0, 2, size=N_FEATURES)
gbest_fitness = -np.inf

# --- Evaluation Function (Reuse the function defined previously) ---
# NOTE: Ensure the evaluate_subset_accuracy function (and X_train/X_test/y_train/y_test)
# from the main script is accessible in the Colab session.
def evaluate_subset_accuracy(individual, X_data_train, X_data_test, y_data_train, y_data_test):
    """Evaluates the fitness (accuracy) of a feature subset."""
    selected = [i for i, bit in enumerate(individual) if bit == 1]
    if len(selected) == 0:
        return 0.0, 0

    X_sel = X_data_train[:, selected]
    X_tst = X_data_test[:, selected]

    from sklearn.linear_model import LogisticRegression
    clf = LogisticRegression(max_iter=300, random_state=42, solver='liblinear')
    clf.fit(X_sel, y_data_train)
    preds = clf.predict(X_tst)
    acc = accuracy_score(y_data_test, preds)
    return acc, len(selected)


# --- BPSO Main Loop (10 Iterations) ---
print("\n" + "="*70)
print(f"🚀 Starting QUICK BPSO Test Run: {QUICK_N_ITERATIONS} Iterations")
print(f"Parameters: Swarm={QUICK_N_PARTICLES}, w={QUICK_W}, c1={QUICK_C1}, c2={QUICK_C2}")
print("="*70)

for iteration in tqdm(range(QUICK_N_ITERATIONS), desc="PSO Quick Iterations"):
    # 1. Evaluate Fitness & Update Bests
    for i in range(QUICK_N_PARTICLES):
        acc, _ = evaluate_subset_accuracy(positions[i], X_train, X_test, y_train, y_test)

        # Update Personal Best (pbest)
        if acc > pbest_fitness[i]:
            pbest_fitness[i] = acc
            pbest_positions[i] = positions[i].copy()

            # Update Global Best (gbest)
            if acc > gbest_fitness:
                gbest_fitness = acc
                gbest_position = positions[i].copy()

    # 2. Update Velocity and Position
    for i in range(QUICK_N_PARTICLES):
        r1 = np.random.rand(N_FEATURES)
        r2 = np.random.rand(N_FEATURES)

        # Velocity Update
        cognitive_vel = QUICK_C1 * r1 * (pbest_positions[i] - positions[i])
        social_vel = QUICK_C2 * r2 * (gbest_position - positions[i])
        velocities[i] = QUICK_W * velocities[i] + cognitive_vel + social_vel
        velocities[i] = np.clip(velocities[i], -QUICK_V_MAX, QUICK_V_MAX)

        # Position Update (Binary Sigmoid)
        prob = 1.0 / (1.0 + np.exp(-velocities[i]))
        rand_nums = np.random.rand(N_FEATURES)
        positions[i] = (rand_nums < prob).astype(int)

    selected_count = int(np.sum(gbest_position))
    tqdm.write(f"  Iter {iteration+1:2d}/{QUICK_N_ITERATIONS} | Best Acc: {gbest_fitness:.4f} | Features: {selected_count}")


# ==============================
# ✅ Final Results for Quick Run
# ==============================
final_acc, final_features_count = evaluate_subset_accuracy(gbest_position, X_train, X_test, y_train, y_test)

print("\n✅ Quick BPSO Run Complete (10 Iterations)!")
print(f"🎯 Final Test Accuracy: {final_acc:.4f}")
print(f"📏 Features Selected: {final_features_count} / {N_FEATURES}")


🚀 Starting QUICK BPSO Test Run: 10 Iterations
Parameters: Swarm=50, w=0.7, c1=1.7, c2=1.3


PSO Quick Iterations:  10%|█         | 1/10 [00:12<01:53, 12.63s/it]

  Iter  1/10 | Best Acc: 0.9701 | Features: 275


PSO Quick Iterations:  20%|██        | 2/10 [00:25<01:39, 12.49s/it]

  Iter  2/10 | Best Acc: 0.9701 | Features: 275


...

  Iter  8/10 | Best Acc: 0.9716 | Features: 252


PSO Quick Iterations:  90%|█████████ | 9/10 [01:59<00:13, 13.66s/it]

  Iter  9/10 | Best Acc: 0.9716 | Features: 252


PSO Quick Iterations: 100%|██████████| 10/10 [02:13<00:00, 13.31s/it]


  Iter 10/10 | Best Acc: 0.9716 | Features: 252

✅ Quick BPSO Run Complete (10 Iterations)!
🎯 Final Test Accuracy: 0.9716
📏 Features Selected: 252 / 512


In [ ]:
# ==============================
# 🧬 Binary Particle Swarm Optimization (BPSO) Core - **REVISED**
# ==============================

# --- PSO Hyperparameters (Based on your Summary Table) ---
N_PARTICLES = 100   # Swarm Size: Increased for a more robust search!
W = 0.7             # Inertia weight: Balances global and local search
C1 = 1.7            # Cognitive coefficient: Attraction to personal best (pbest)
C2 = 1.3            # Social coefficient: Attraction to global best (gbest)
V_MAX = 6.0         # Maximum velocity (Kept for stability in binary PSO)

# --- Evaluation Function (Remains the same, but defined again for completeness) ---
def evaluate_subset_accuracy(individual, X_data_train, X_data_test, y_data_train, y_data_test):
    """Evaluates the fitness (accuracy) of a feature subset."""
    selected = [i for i, bit in enumerate(individual) if bit == 1]
    if len(selected) == 0:
        return 0.0, 0 # Return accuracy and feature count

    # Select features
    X_sel = X_data_train[:, selected]
    X_tst = X_data_test[:, selected]

    # Train and evaluate a simple classifier
    # solver='liblinear' is generally good for small datasets and binary classification
    # Max_iter is kept at 300 for faster evaluation within the inner loop
    clf = LogisticRegression(max_iter=300, random_state=42, solver='liblinear')
    clf.fit(X_sel, y_data_train)
    preds = clf.predict(X_tst)
    acc = accuracy_score(y_data_test, preds)
    return acc, len(selected)


# --- BPSO Runner Function ---
def run_bpso(n_iterations, X_train, X_test, y_train, y_test, n_features):
    """Initializes and runs the BPSO algorithm for a given number of iterations."""

    # Initialization
    # Initialize positions (particles) randomly
    positions = np.random.randint(0, 2, size=(N_PARTICLES, n_features))
    velocities = np.zeros((N_PARTICLES, n_features))

    # Initialize personal bests (pbest)
    pbest_positions = positions.copy()
    pbest_fitness = np.full(N_PARTICLES, -np.inf)

    # Initialize global best (gbest)
    gbest_position = np.random.randint(0, 2, size=n_features)
    gbest_fitness = -np.inf

    # Storage for tracking results
    history = []

    print(f"\n🚀 Starting BPSO for {n_iterations} Iterations...")
    # Using tqdm for the outer loop to track generation progress smoothly
    for iteration in tqdm(range(n_iterations), desc="PSO Iterations"):

        # 1. Evaluate Fitness & Update Bests
        for i in range(N_PARTICLES):
            acc, _ = evaluate_subset_accuracy(positions[i], X_train, X_test, y_train, y_test)

            # Update Personal Best (pbest)
            if acc > pbest_fitness[i]:
                pbest_fitness[i] = acc
                pbest_positions[i] = positions[i].copy()

                # Update Global Best (gbest)
                if acc > gbest_fitness:
                    gbest_fitness = acc
                    gbest_position = positions[i].copy()

        # 2. Update Velocity and Position
        for i in range(N_PARTICLES):
            r1 = np.random.rand(n_features)
            r2 = np.random.rand(n_features)

            # Calculate new velocity (Continuous update)
            cognitive_vel = C1 * r1 * (pbest_positions[i] - positions[i])
            social_vel = C2 * r2 * (gbest_position - positions[i])

            velocities[i] = W * velocities[i] + cognitive_vel + social_vel

            # Clamp velocity
            velocities[i] = np.clip(velocities[i], -V_MAX, V_MAX)

            # Calculate Sigmoid (S-shaped transfer function)
            prob = 1.0 / (1.0 + np.exp(-velocities[i]))

            # Update position (Binary update)
            rand_nums = np.random.rand(n_features)
            positions[i] = (rand_nums < prob).astype(int)

        # Track results for this iteration
        selected_count = int(np.sum(gbest_position))
        history.append((iteration + 1, gbest_fitness, selected_count))

        # Print update every 10 iterations or at the end
        if (iteration + 1) % 10 == 0 or iteration == n_iterations - 1:
            tqdm.write(f"  Iter {iteration+1:3d}/{n_iterations} | Best Acc: {gbest_fitness:.4f} | Features: {selected_count}")


    # Final Evaluation
    final_acc, final_features_count = evaluate_subset_accuracy(gbest_position, X_train, X_test, y_train, y_test)

    print(f"\n✅ BPSO Run Complete ({n_iterations} Iterations)!")
    print(f"🎯 Final Test Accuracy: {final_acc:.4f}")
    print(f"📏 Features Selected: {final_features_count} / {n_features}")

    return final_acc, final_features_count, gbest_position, history


# ==============================
# 🔁 Execute Runs for 25, 50, and 100 Iterations
# ==============================
ITERATIONS_TO_TEST = [25, 50, 100] # Based on your Summary Table generations/iterations
all_results = {}

# Ensure data split is available from the initial part of the code
if 'X_train' not in locals():
    # Placeholder for the missing first part of the code execution
    print("\n❌ ERROR: Data (X_train, X_test, etc.) is not defined. Please run the setup and feature extraction part first!")
else:
    for NGEN in ITERATIONS_TO_TEST:
        print("\n" + "="*70)
        print(f"Starting experiment for PSO Iterations = {NGEN}")
        print(f"Hyperparameters: Swarm={N_PARTICLES}, w={W}, c1={C1}, c2={C2}")
        print("="*70)

        final_acc, final_features_count, best_subset, history = run_bpso(
            NGEN, X_train, X_test, y_train, y_test, N_FEATURES
        )

        all_results[NGEN] = {
            "accuracy": final_acc,
            "features": final_features_count,
            "subset": best_subset,
            "history": history
        }

    # ==============================
    # 📊 Summary of Results
    # ==============================
    print("\n" + "#"*70)
    print("### 🏆 Final Comparison of PSO Convergence Across Iterations ###")
    print("#"*70)

    for NGEN, result in all_results.items():
        print(f"  ▶️ {NGEN} Iterations:")
        print(f"     Accuracy: {result['accuracy']:.4f}")
        print(f"     Features: {result['features']} / {N_FEATURES}\n")


Starting experiment for PSO Iterations = 25
Hyperparameters: Swarm=100, w=0.7, c1=1.7, c2=1.3

🚀 Starting BPSO for 25 Iterations...


PSO Iterations:  40%|████      | 10/25 [04:18<06:29, 25.97s/it]

  Iter  10/25 | Best Acc: 0.9739 | Features: 270


PSO Iterations:  80%|████████  | 20/25 [08:29<02:04, 24.91s/it]

  Iter  20/25 | Best Acc: 0.9747 | Features: 258


...

  Iter  80/100 | Best Acc: 0.9755 | Features: 242


PSO Iterations:  90%|█████████ | 90/100 [36:28<04:02, 24.28s/it]

  Iter  90/100 | Best Acc: 0.9755 | Features: 242


PSO Iterations: 100%|██████████| 100/100 [40:31<00:00, 24.31s/it]


  Iter 100/100 | Best Acc: 0.9770 | Features: 258

✅ BPSO Run Complete (100 Iterations)!
🎯 Final Test Accuracy: 0.9770
📏 Features Selected: 258 / 512

######################################################################
### 🏆 Final Comparison of PSO Convergence Across Iterations ###
######################################################################
  ▶️ 25 Iterations:
     Accuracy: 0.9747
     Features: 258 / 512

  ▶️ 50 Iterations:
     Accuracy: 0.9755
     Features: 242 / 512

  ▶️ 100 Iterations:
     Accuracy: 0.9770
     Features: 258 / 512

